In [ ]:
# Read the SQL view
df_dim_date = spark.sql("""
SELECT DISTINCT 
    a.year,
    a.month,
    CAST(CONCAT(a.year, '-', lpad(a.month, 2, '0'), '-01') AS DATE) AS first_day
FROM (
    SELECT DISTINCT month, year FROM fgc_gld_redata_demanda_month
    UNION
    SELECT DISTINCT month, year FROM fgc_gld_redata_generacion_estructura_month
    UNION
    SELECT DISTINCT month,year FROM gld_demanda_mensual --METEO INFO
) AS a
""")

df_dim_date.show()

# Write it as a managed Delta table in the same Lakehouse
(df_dim_date.write
    .format("delta")
    .mode("overwrite")  # or 'append'
    .option("overwriteSchema", "true")
    .saveAsTable("fgc_dim_date"))



StatementMeta(, bd0cfe30-84bb-457d-975d-3e2d91eda0c4, 9, Finished, Available, Finished)

+----+-----+----------+
|year|month| first_day|
+----+-----+----------+
|2023|   11|2023-11-01|
|2023|    8|2023-08-01|
|2023|    4|2023-04-01|
|2024|    3|2024-03-01|
|2025|    5|2025-05-01|
|2024|    8|2024-08-01|
|2024|   10|2024-10-01|
|2024|   12|2024-12-01|
|2024|    2|2024-02-01|
|2025|    7|2025-07-01|
|2025|    9|2025-09-01|
|2024|   11|2024-11-01|
|2023|    1|2023-01-01|
|2023|   12|2023-12-01|
|2025|    3|2025-03-01|
|2024|    4|2024-04-01|
|2023|    2|2023-02-01|
|2024|    1|2024-01-01|
|2024|    6|2024-06-01|
|2024|    5|2024-05-01|
+----+-----+----------+
only showing top 20 rows



In [ ]:
#ESIOS DATE DIM
# Read the SQL view
df_dim_date = spark.sql("""
SELECT DISTINCT 
    a.fecha as esios_date, a.esios_month, a.esios_year as esios_year
FROM (
    SELECT DISTINCT fecha, month(fecha) as esios_month, year(fecha) as esios_year  FROM gld_esios_mercados_demanda_real_day
    UNION
    SELECT DISTINCT fecha, month(fecha) as esios_month, year(fecha) as esios_year  FROM gld_esios_mercados_mercado_spot_day
    UNION
    SELECT DISTINCT fecha, month(fecha) as esios_month, year(fecha) as esios_year FROM gld_esios_mercados_mercado_spot_diario_espana_day
) AS a
""")

df_dim_date.show()

# Write it as a managed Delta table in the same Lakehouse
(df_dim_date.write
    .format("delta")
    .mode("overwrite")  # or 'append'
    .option("overwriteSchema", "true")
    .saveAsTable("esios_dim_date"))

StatementMeta(, c7e5765e-f813-4e5d-9bd3-bf14f2e83f3c, 6, Finished, Available, Finished)

+-------------------+-----------+----------+
|         esios_date|esios_month|esios_year|
+-------------------+-----------+----------+
|2023-07-08 22:00:00|          7|      2023|
|2024-02-06 23:00:00|          2|      2024|
|2024-07-17 22:00:00|          7|      2024|
|2024-10-30 23:00:00|         10|      2024|
|2025-10-13 00:35:00|         10|      2025|
|2025-10-13 04:10:00|         10|      2025|
|2025-10-13 16:05:00|         10|      2025|
|2023-09-18 22:00:00|          9|      2023|
|2024-06-07 22:00:00|          6|      2024|
|2024-06-22 22:00:00|          6|      2024|
|2024-06-30 22:00:00|          6|      2024|
|2024-08-22 22:00:00|          8|      2024|
|2025-06-12 22:00:00|          6|      2025|
|2025-07-29 22:00:00|          7|      2025|
|2025-10-13 15:40:00|         10|      2025|
|2025-10-13 21:10:00|         10|      2025|
|2023-10-15 22:00:00|         10|      2023|
|2023-10-22 22:00:00|         10|      2023|
|2025-02-08 23:00:00|          2|      2025|
|2025-10-1

In [ ]:
# Read the SQL view
df_dim_geo = spark.sql("SELECT DISTINCT a.geo_id, UPPER(LEFT(a.geo_name,3)) as ccaa_iniciales, a.geo_name,CONCAT(CAST(a.geo_id AS VARCHAR(50)),' - ', a.geo_name) as cod_ccaa FROM ( SELECT DISTINCT geo_id, geo_name FROM fgc_gld_redata_demanda_month  UNION  SELECT DISTINCT geo_id, geo_name FROM fgc_gld_redata_generacion_estructura_month) AS a")

# Write it as a managed Delta table in the same Lakehouse
(df_dim_geo.write
    .format("delta")
    .mode("overwrite")  # or 'append'
    .option("overwriteSchema", "true")
    .saveAsTable("fgc_dim_geography"))

StatementMeta(, bd0cfe30-84bb-457d-975d-3e2d91eda0c4, 4, Finished, Available, Finished)

In [ ]:
# Read the SQL view
df_dim_technology = spark.sql("SELECT DISTINCT 	UPPER(LEFT(a.series_type,3)) as tech_iniciales, (CASE WHEN a.series_type = 'Residuos no renovables' AND a.series_attribute_type = 'Renovable' THEN 'Residuos' ELSE a.series_type END) as tecnologia, 	a.series_attribute_type as tech_type FROM fgc_gld_redata_generacion_estructura_month as a")

# Write it as a managed Delta table in the same Lakehouse
(df_dim_technology.write
    .format("delta")
    .mode("overwrite")  # or 'append'
    .option("overwriteSchema", "true")
    .saveAsTable("fgc_dim_technology"))

StatementMeta(, bd0cfe30-84bb-457d-975d-3e2d91eda0c4, 7, Finished, Available, Finished)